# Variables

In [7]:
OPENAI_API_KEY='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'
FIRECRAWL_API_KEY='fc-e36282d62d1c4ee8971ec0649f3862c7'
BASE_URL='https://api.groq.com/openai/v1'
MODEL="meta-llama/llama-4-scout-17b-16e-instruct"

# Import Modules

In [8]:
import sys
sys.path.append('..')
sys.path.append('.')
print(sys.path)
from tools.fetching_description_from_huggingface import fetching_description_from_huggingface
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_tool_calling_agent, AgentExecutor
from pydantic import BaseModel, Field
from langchain.tools import Tool
from firecrawl import FirecrawlApp
from langchain_core.output_parsers import PydanticOutputParser
import pandas as pd
import pprint

['/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/mnt/g/Ai agents course/LangGraph/venv/lib/python3.12/site-packages', '..', '.', '..', '.', '..', '.']


# Testing

In [9]:
df_task = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Public task.csv")
df_model = pd.read_csv("data/ISE - AutoCode Challenge 2 Public - Model zoo.csv")

In [10]:
df_task

,STT,Task,Validation data
0,1,"""Bối cảnh của vấn đề:\r\nNhận dạng chữ viết ta...",Link
1,2,Bối cảnh của vấn đề:\r\nTrong thời đại thông t...,Link
2,3,Bối cảnh của vấn đề:\r\nĐây là một nhiệm vụ hồ...,Link
3,4,Bối cảnh của vấn đề:\r\nĐiều quan trọng là các...,Link
4,5,Bối cảnh của vấn đề:\r\nTrong nhiều ứng dụng t...,Link
5,6,🐾 Story: Alice’s Magical Pet Photo Box\r\nAlic...,Link


In [11]:
task = df_task["Task"][5]

In [12]:
task

'🐾 Story: Alice’s Magical Pet Photo Box\r\nAlice is an animal lover who adores collecting pictures of pets. In her room, she keeps a magical box filled with hundreds of photos of dogs and cats that she has taken or found from all over the internet.\r\n\r\nBut there’s a little problem...\r\n\r\nOver time, Alice’s collection has gotten messy. Her box now contains a mix of dog photos, cat photos, and even some blurry or unrecognizable images. She needs a smart system to help her sort through the mess and organize everything again.\r\n\r\n🧠 Your Mission\r\nYou’re building an intelligent system that can analyze each image in Alice’s magical box and decide what it contains. Each image will fall into one of the following categories:\r\n\r\n🐶 If the image contains dogs:\r\n\r\nThere may be one or more dogs in the photo.\r\n\r\nYour system should detect all visible dogs, identify their breed, and return the most frequent breed in the image.\r\n\r\n🐱 If the image contains a single cat:\r\n\r\nLo

In [13]:
def format_model_list(df):
    model_strings = []
    for i, row in df.iterrows():
        desc = str(row[1]).replace("{", "{{").replace("}", "}}")
        url = str(row[2])
        model_strings.append(f"{i + 1} - {desc} [More info]({url})")
    return "\n".join(model_strings)


## Tools

In [14]:
def web_scraper(url: str) -> str:
    """Get more details about the model using the model url"""
    # scraper = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
    # scrape_status = scraper.scrape_url(
    #     url,
    #     formats=['markdown']
    # )
    # return scrape_status.markdown
    result = fetching_description_from_huggingface(url)
    return result

In [15]:
class WebScraperParameters(BaseModel):
    url: str

In [16]:
res = web_scraper('https://huggingface.co/thanhtlx/image_classification_01')

In [17]:
tools = [
    Tool(
        name="web_scrapper",
        func=web_scraper,
        description="Use to get more detail about the model",
        args_schema=WebScraperParameters
    )
]

## Model

In [18]:
llm = ChatOpenAI( 
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0.7
)

In [19]:
class Tasks(BaseModel):
    subtask_one: str = Field(
        ...,
        description="Detailed description of task 1"
    )
    subtask_two: str = Field(
        ...,
        description="Detailed description of task 2"
    )
    subtask_three: str = Field(
        ...,
        description="Detailed description of task 3"
    )

In [20]:
class AgentState(TypedDict):
    task: HumanMessage | None
    data: str | None
    messages: Annotated[BaseMessage, add_messages]
    subtasks: Tasks

In [21]:
def classify_agent(state: AgentState) -> AgentState:
    structured_llm = llm.with_structured_output(Tasks)
    system_prompt = """
                    You are an expert in task formulation. Given a machine learning task, you will
                    help divide the task into these subtasks:
                    - Subtask1: for classifying the problem type and task description
                    - Subtask2: for identifying the input of the problem
                    - Subtask3: for identifying the output of the problem
                    describe the each subtask as specifically as possible
                    """
    messages = [SystemMessage(content=system_prompt)]
    messages.append(state['task'])
    result = structured_llm.invoke(messages)
    state['subtasks'] = result
    return state

In [22]:
state = {
    "task": HumanMessage(content=task),
}

In [23]:
state = classify_agent(state)

# Modeling Agent

In [24]:
class ModelInfo(BaseModel):
    model: str = Field(..., description="Model name")
    url: str = Field(..., description="URL to the model")

class ModelSelection(BaseModel):
    models: List[ModelInfo] = Field(..., description="List of available models with name and URL")

In [25]:
model_selection_parser = PydanticOutputParser(pydantic_object=ModelSelection)

In [26]:
def model_selection_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the suitable model for a given task.
                    Given:
                    - A user description about the task
                    - A list of available model (names and links)
                    Your job is to choose ALL suitable models provided in the model list for user's specific tasks.
                    Return the answer in the format
                    {structured_output}
                    Here are the list of model:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}")
    ]).partial(structured_output=model_selection_parser.get_format_instructions())

    chain = prompt | llm | model_selection_parser

    result = chain.invoke({"input": task, "model_list": format_model_list(df_model)})
    return result

In [27]:
res = model_selection_agent(state)

/tmp/ipykernel_10912/7332665.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  desc = str(row[1]).replace("{", "{{").replace("}", "}}")
/tmp/ipykernel_10912/7332665.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  url = str(row[2])


In [28]:
pprint.pprint(res.models)

[ModelInfo(model='Phân loại hình ảnh giống chó', url='https://huggingface.co/wesleyacheng/dog-breeds-multiclass-image-classification-with-vit'),
 ModelInfo(model='Nhận diện cảm xúc mèo', url='https://huggingface.co/semihdervis/cat-emotion-classifier'),
 ModelInfo(model='Phân loại hình ảnh', url='https://huggingface.co/zhaospei/Model_7')]


In [29]:
def model_selection_using_tool_agent(state: AgentState):
    system_prompt = """
                    You are a machine learning expert assigned to select the **Best-fit model** for a given task.
                    Given:
                    - A user task description
                    - A list of available models (names and url)
                    - A tool that fetches model details from a provided URL (it help gathers model description, input/output format, code sample usage)

                    Your job is to:
                    1. Use the **provided tool** to retrieve real details about each candidate model:
                        - ✅ Model description
                        - 📥 Input format
                        - 📤 Output format
                        - 🛠️ Library requirements
                        - 🧪 Code sample (usage code snippet)
                    2. Select the best model for user given task based on the data that the tool gives you.

                    ⚠️ VERY IMPORTANT RULES

                    - ❌ DO NOT assume or invent any part of the model's description, input/output format, requirements or code
                    - ❌ DO NOT generate fake code or use your own knowledge about the model
                    - ✅ ONLY use the **actual output** returned from the tool
                    - ✅ Include tool content in your final answer exactly as returned (especially code)

                    ## 🧠 Output Structure (Final Answer)

                    After retrieving tool results, choose best-fit model for the task (You may need to choose more than one model for some tasks), only choose the model 
                    that strictly relevant to the task and output the model result in the following format:

                    **✅ Model name and link**  
                    `<model name>` — `<link>`

                    **📝 Description (from tool):**  
                    <model description>

                    **📥 Input format (from tool):**  
                    <description of expected input>

                    **📤 Output format (from tool):**  
                    <description of model output>

                    **🛠️ Library Requirements (from tool) **
                    <requirements to use the model>

                    **🧪 Example code (from tool):**  
                    ```python
                    <exact code snippet from tool>

                    Here are the list of models and there corresponding URL:
                    {model_list}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("ai", "{agent_scratchpad}"),
        ("human", "{input}")
    ])

    agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    task_input = state["task"].content if isinstance(state["task"], SystemMessage) else state["task"]

    # result = executor.invoke({"input": task_input})
    result = executor.invoke({"input": task_input, "model_list": res.models})

    return {"output": result["output"]}

In [30]:
res2 = model_selection_using_tool_agent(state)



> Entering new AgentExecutor chain...



Invoking: `web_scrapper` with `{'url': 'https://huggingface.co/wesleyacheng/dog-breeds-multiclass-image-classification-with-vit'}`
responded: To solve this task, I need to analyze the requirements and select the best-fit model for image classification and emotion detection. 

The task requires two main functionalities: 
1. Detecting dogs in an image, identifying their breed, and returning the most frequent breed.
2. Detecting cats in an image and determining their emotion.

Given the models:
- 'Phân loại hình ảnh giống chó' (https://huggingface.co/wesleyacheng/dog-breeds-multiclass-image-classification-with-vit)
- 'Nhận diện cảm xúc mèo' (https://huggingface.co/semihdervis/cat-emotion-classifier)
- 'Phân loại hình ảnh' (https://huggingface.co/zhaospei/Model_7)

I will first gather detailed information about the 'Phân loại hình ảnh giống chó' model.




Model made by notebook first posted in my Kaggle.
Model Motivation
Recently, someone asked me if you can classify dog images into thei

In [31]:
pprint.pprint(res2['output'])

('## ✅ Model1: Phân loại hình ảnh giống chó — '
 '<https://huggingface.co/wesleyacheng/dog-breeds-multiclass-image-classification-with-vit>\n'
 '\n'
 '**📝 Description (from tool):**  \n'
 'This model is finetuned using the Google Vision Transformer '
 '(vit-base-patch16-224-in21k) on the Stanford Dogs dataset in Kaggle to '
 'classify dog images into120 types of dog breeds.\n'
 '\n'
 '**📥 Input format (from tool):**  \n'
 'The model expects an image input.\n'
 '\n'
 '**📤 Output format (from tool):**  \n'
 'The model predicts one of the120 Stanford dog breeds classes.\n'
 '\n'
 '**🛠️ Library Requirements (from tool) **  \n'
 '- Transformers\n'
 '- Pytorch\n'
 '- PIL\n'
 '\n'
 '**🧪 Example code (from tool):**  \n'
 '```python\n'
 'from transformers import AutoImageProcessor, '
 'AutoModelForImageClassification\n'
 'import PIL\n'
 'import requests\n'
 '\n'
 'url = "https://upload.wikimedia.org/wikipedia/commons/5/55/Beagle_600.jpg"\n'
 'image = PIL.Image.open(requests.get(url, stream=True

## Output Parser Agent

In [32]:
class ModelingOutput(BaseModel):
    model_description: str = Field(
        ...,
        description="The detailed description of the model."
    )
    model_input_format: str = Field(
        ...,
        description="Model's detailed input format"
    )
    model_output_format: str = Field(
        ...,
        description="Model's detailed output format"
    )
    model_requirements: str = Field(
        ...,
        description="Requirement libraries need to install to use the model."
    )
    model_sample_code: str = Field(
        ...,
        description="Example code to use the model"
    )

class ModelList(BaseModel):
    models: List[ModelingOutput] = Field(
        ...,
        description="List of suitable model for given task"
    )

In [33]:
parser = PydanticOutputParser(pydantic_object=ModelList)

In [34]:
output_parser_llm = ChatOpenAI(
    base_url=BASE_URL,
    model=MODEL,
    api_key=OPENAI_API_KEY,
    temperature=0
)

In [35]:
def output_parser_agent(state: AgentState):
    system_prompt = """
                    You are a smart AI tasked with extracting structured technical details about a machine learning model from a reasoning result.
                    You are given a detailed text description about a model (or a list of model).
                    Your goal is to fill the following fields based on the text:

                    - `model_description`: A detailed explanation of what the model is and what it does.
                    - `model_input_format`: A detailed description of the model's input format, including dimensions, data types, and expected preprocessing if mentioned.
                    - `model_output_format`: A detailed description of the output format including dimensions, data types, label name and its meaning.
                    - `model_requirements`: A detailed description about the requirements needed to be sastified in order to use the model
                    - `model_sample_code`: A sample code on how to use the model.

                    You MUSTN'T create any data on your own, only using the data provided in the text.

                    Return the data as a JSON object matching the following structure:
                    {formatted_output}
                    """
    prompt = ChatPromptTemplate.from_messages([
        ('system', system_prompt),
        ('human', "{input}")
    ]).partial(formatted_output=parser.get_format_instructions())
    chain = prompt | llm | parser
    response = chain.invoke({"input": res2['output']})
    return response

In [36]:
ans = output_parser_agent(state)

In [37]:
pprint.pprint(res2['output'])

('## ✅ Model1: Phân loại hình ảnh giống chó — '
 '<https://huggingface.co/wesleyacheng/dog-breeds-multiclass-image-classification-with-vit>\n'
 '\n'
 '**📝 Description (from tool):**  \n'
 'This model is finetuned using the Google Vision Transformer '
 '(vit-base-patch16-224-in21k) on the Stanford Dogs dataset in Kaggle to '
 'classify dog images into120 types of dog breeds.\n'
 '\n'
 '**📥 Input format (from tool):**  \n'
 'The model expects an image input.\n'
 '\n'
 '**📤 Output format (from tool):**  \n'
 'The model predicts one of the120 Stanford dog breeds classes.\n'
 '\n'
 '**🛠️ Library Requirements (from tool) **  \n'
 '- Transformers\n'
 '- Pytorch\n'
 '- PIL\n'
 '\n'
 '**🧪 Example code (from tool):**  \n'
 '```python\n'
 'from transformers import AutoImageProcessor, '
 'AutoModelForImageClassification\n'
 'import PIL\n'
 'import requests\n'
 '\n'
 'url = "https://upload.wikimedia.org/wikipedia/commons/5/55/Beagle_600.jpg"\n'
 'image = PIL.Image.open(requests.get(url, stream=True

In [38]:
for i in range(len(ans.models)):
    print(f"Model {i}:")
    pprint.pprint(ans.models[i].model_description)
    pprint.pprint(ans.models[i].model_input_format)
    pprint.pprint(ans.models[i].model_output_format)
    pprint.pprint(ans.models[i].model_requirements)
    pprint.pprint(ans.models[i].model_sample_code)

Model 0:
('This model is finetuned using the Google Vision Transformer '
 '(vit-base-patch16-224-in21k) on the Stanford Dogs dataset in Kaggle to '
 'classify dog images into 120 types of dog breeds.')
'The model expects an image input.'
'The model predicts one of the 120 Stanford dog breeds classes.'
'Transformers, Pytorch, PIL'
('from transformers import AutoImageProcessor, '
 'AutoModelForImageClassification\n'
 'import PIL\n'
 'import requests\n'
 '\n'
 'url = "https://upload.wikimedia.org/wikipedia/commons/5/55/Beagle_600.jpg"\n'
 'image = PIL.Image.open(requests.get(url, stream=True).raw)\n'
 'image_processor = '
 'AutoImageProcessor.from_pretrained("wesleyacheng/dog-breeds-multiclass-image-classification-with-vit")\n'
 'model = '
 'AutoModelForImageClassification.from_pretrained("wesleyacheng/dog-breeds-multiclass-image-classification-with-vit")\n'
 '\n'
 'inputs = image_processor(images=image, return_tensors="pt")\n'
 'outputs = model(**inputs)\n'
 'logits = outputs.logits\n'
 